In [1]:
import sys
import os
project_dir = os.path.dirname(os.getcwd())
sys.path.append(project_dir) #notebooks can't look into source unless you append the parent project directory to the path
print(project_dir)
sys.path.append('../src')

/home/ubuntu/courtney/usal_echo


In [2]:
import pandas as pd
import datetime

In [3]:
from d04_segmentation.create_probabilities_textfile_testing import create_probabilities_textfile_testing

/home/ubuntu/anaconda3/envs/usal_echo/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [4]:
from src.d00_utils.db_utils import dbReadWriteViews

In [5]:
io_views = dbReadWriteViews()

In [6]:
io_views.list_tables()

['machines_all_bmi', 'instances_with_labels_scrappy100', 'machines_new_bmi', 'instances_unlabeled', 'frames_sorted_by_views_temp', 'frames_w_labels', 'instances_unique_master_list', 'instances_w_labels_train', 'instances_w_conflicts', 'instances_w_labels_test', 'instances_w_labels', 'instances_w_labels_train_downsampleby5', 'instances_w_labels_test_downsampleby5', 'chords_by_volume_mask', 'frames_by_volume_mask', 'output_m1_test1000_aug8']


In [7]:
instance_id_list = io_views.get_table('instances_w_labels_test_downsampleby5') 

In [8]:
print(instance_id_list)

                                         sopinstanceuid  filename  \
0     1.2.840.113663.1500.1.365900332.3.8.20130125.1...  1LHFYKEZ   
1     1.2.840.113663.1500.1.365900332.3.1.20130321.1...  43IX4IH1   
2     1.2.840.113663.1500.1.365900332.3.11.20130611....  7SQZPLEJ   
3     1.2.840.113663.1500.1.365900332.3.1.20131003.1...  CSTYB214   
4     1.2.840.113663.1500.1.365900332.3.8.20131120.1...  EAKXA40T   
...                                                 ...       ...   
1020  1.2.840.113663.1500.1.402357036.3.1.20181130.1...  EF74LO2Z   
1021  1.2.840.113663.1500.1.365900332.3.8.20181205.1...  FDNHKCYK   
1022  1.2.840.113663.1500.1.406830382.3.10.20181205....  FDNHO33T   
1023  1.2.840.113663.1500.1.402357036.3.1.20181205.1...  FDNHPQ07   
1024  1.2.840.113663.1500.1.365900332.3.7.20181205.1...  FDNHQ1M9   

      instanceidk  view  studyidk  
0          779540   a4c     43763  
1          804660  plax     45076  
2          841585   a4c     46919  
3          892588  plax    

In [9]:
instance_id_list = instance_id_list['instanceidk']

In [31]:
# 1. Gets frames_by_volume_mask view
io_views = dbReadWriteViews()
frames_by_volume_mask = io_views.get_table("frames_by_volume_mask")
print('frames_by_volume_mask size: {}'.format(frames_by_volume_mask.shape))
#frames_by_volume_mask = frames_by_volume_mask[
#    frames_by_volume_mask["view_exists"] == True
#]
instances_w_labels_test = io_views.get_table("instances_w_labels_test_downsampleby5")
print('instance_list size: {}'.format(instances_w_labels_test.shape))


frames_by_volume_mask size: (72527, 9)
instance_list size: (1025, 5)


In [32]:
# take out frames and drop duplicates
frames_by_volume_mask = frames_by_volume_mask.drop('frame', axis=1)
frames_by_volume_mask = frames_by_volume_mask.drop_duplicates()
frames_by_volume_mask.shape

(72527, 8)

In [25]:
frames_by_volume_mask.columns
print(frames_by_volume_mask.head(1))

   instanceidk  indexinmglist  studyidk  frame   view_name view_exists  \
0        57164              1      2882     15  a4c_ven_ed           t   

  view_only ventricle_only cycle_only  
0       a4c            ven         ed  


In [26]:
frames_by_volume_mask.rename(columns={'view': 'full_view', 'view_only':'view'}, inplace=True)

In [27]:
instances_w_labels_test.columns
print(instances_w_labels_test.head(1))

                                      sopinstanceuid  filename  instanceidk  \
0  1.2.840.113663.1500.1.365900332.3.8.20130125.1...  1LHFYKEZ       779540   

  view  studyidk  
0  a4c     43763  


In [28]:
# 2. Merges tables
df_1 = pd.merge(
    instances_w_labels_test,
    frames_by_volume_mask,
    how="inner",
    on=["instanceidk", "studyidk", "view"],
)
print(df_1.shape)
print(df_1.count())

(1157, 11)
sopinstanceuid    1157
filename          1157
instanceidk       1157
view              1157
studyidk          1157
indexinmglist     1157
frame             1157
view_name         1157
view_exists       1157
ventricle_only    1157
cycle_only        1157
dtype: int64


In [ ]:
# 3. Filters the dataframe by the param: instanceid
df_2 = df_1[df_1["instanceidk"].isin(instance_id_list)]
print('df_2: {}'.format(df_2.shape))
del df_1

print(df_2.shape)
print(df_2.head())

In [ ]:
# 4. Create the probabilities table
prob_tb = pd.DataFrame(
    columns=[
        "study",
        "image",
        "plax_far",
        "plax_plax",
        "plax_laz",
        "psax_az",
        "psax_mv",
        "psax_pap",
        "a2c_lvocc_s",
        "a2c_laocc",
        "a2c",
        "a3c_lvocc_s",
        "a3c_laocc",
        "a3c",
        "a4c_lvocc_s",
        "a4c_laocc",
        "a4c",
        "a5c",
        "other",
        "rvinf",
        "psax_avz",
        "suprasternal",
        "subcostal",
        "plax_lac",
        "psax_apex",
    ]
)

for i in df_2.index:
    prob_tb.at[i, "study"] = "/home/ubuntu/data/01_raw/test_downsampleby5"
    filename = df_2.at[i, "filename"]
    studyidk = df_2.at[i, "studyidk"]
    prob_tb.at[i, "image"] = (
        "a_" + str(studyidk).strip() + "_" + str(filename).strip() + ".dcm"
    )
    if df_2.at[i, "view_only"] == "a4c":
        prob_tb.at[i, "a4c"] = 0.9898
    elif df_2.at[i, "view_only"] == "a2c":
        prob_tb.at[i, "a2c"] = 0.9898

df_3 = prob_tb.fillna(0)

print(df_3.shape)

In [ ]:
# 5. Writes the probabilities textfile to data/d04_segmentation folder
project_dir = os.path.dirname(os.getcwd())
data_dir = os.path.join(project_dir, "data", "d04_segmentation")

if not os.path.exists(data_dir):
    os.makedirs(data_dir)

time_stamp = datetime.date.today()
file_name = "view_probabilities_test" + str(time_stamp) + ".txt"
data_path = os.path.join(data_dir, file_name)

df_3.to_csv(data_path, index=None, header=True, sep="\t")